<a href="https://colab.research.google.com/github/frag30041997/jupyter-one/blob/master/Cours_DA3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DA regression (prediction valeurs continues?)

In [17]:
from sklearn.linear_model import LogisticRegression
import seaborn as sns; sns.set
from sklearn.model_selection import train_test_split

def accuracy(preds, target):
  M = target.shape[0] #nb d'ex
  total_correct = (preds == target).sum()
  accuracy = (total_correct/M)
  return accuracy

iris = sns.load_dataset('iris')

iris.shape


(150, 5)

In [12]:
y = iris.species.astype('category').cat.codes
vocabulaire = iris.species.astype('category').cat.categories
x = iris.drop('species',axis='columns')
x.shape,y.shape

((150, 4), (150,))

In [43]:
splits = train_test_split(x,y, train_size=0.5,stratify=y,random_state=2)
x_train = splits[0]
x_test = splits[1]
y_train = splits[2]
y_test = splits[3]
print(x_train.shape)
print(x_test.shape)

(75, 4)
(75, 4)


In [44]:
model = LogisticRegression()
model.fit(x_train, y_train)
pred = model.predict(x_test)
test = model.predict_proba(x_test)[1, :]
print(test)

[0.00097676 0.22516097 0.77386227]


In [45]:
accuracy(pred,y_test)

1.0

In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split as TTS
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier as KNN

In [47]:
df = sns.load_dataset('iris')

In [48]:
#Isolation
#target_feature_name = 'target_feature'
target_feature_name = 'species'
Y = df[target_feature_name]
X = df.drop(columns=target_feature_name)

In [49]:
#TTS
X_tr, X_te, Y_tr, Y_te = TTS(X, Y, stratify=Y, random_state=314)

In [50]:
#Pipeline
pipeline_details = [('PCA', PCA(random_state=42)),
                    ('KNN', KNN())]
pipeline = Pipeline(steps=pipeline_details)

In [51]:
#GridSearchCV
print([i for i in range(1, X_tr.shape[1] + 1)])
print([i for i in range(1, 21 + 1, 2)])

hyperparameters = {}
hyperparameters['PCA__n_components'] = [i for i in range(1, X_tr.shape[1]+1)]
hyperparameters['KNN__n_neighbors']  = [i for i in range(1, 21 +1, 2)]

print(hyperparameters)

hyperparameter_search = GridSearchCV(pipeline,
                                     hyperparameters,
                                     scoring='accuracy', 
                                     cv=5)

hyperparameter_search.fit(X_tr, Y_tr)

print('Meilleur score : {:.5f}'.format(hyperparameter_search.best_score_))
print('Meilleur paramètres : {}'.format(hyperparameter_search.best_params_))

[1, 2, 3, 4]
[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]
{'PCA__n_components': [1, 2, 3, 4], 'KNN__n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]}
Meilleur score : 0.99091
Meilleur paramètres : {'KNN__n_neighbors': 15, 'PCA__n_components': 4}


In [52]:
#PCA
N=hyperparameter_search.best_params_['PCA__n_components']
pca = PCA(n_components=N)
pca.fit(X_tr)
X_tr_PCA = pca.transform(X_tr)
X_te_PCA = pca.transform(X_te)

In [53]:
#KNN
N=hyperparameter_search.best_params_['KNN__n_neighbors']
knn = KNN(n_neighbors=N)
knn.fit(X_tr_PCA, Y_tr)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=15, p=2,
                     weights='uniform')

In [54]:
#Accuracy
preds = knn.predict(X_te)
accuracy = lambda p, y : (p==y).sum()/len(y)
print('Accuracy : {}'.format(accuracy(preds, Y_te)))

Accuracy : 0.3157894736842105


In [55]:
#Chargement
#df = pd.read_csv("data/filename.csv")
df = pd.DataFrame(X_te_PCA)

In [56]:
#KNN
preds = knn.predict(df)
df['Predictions'] = preds

In [57]:
df = df.drop(columns='Predictions')

In [58]:
unknow_iris = df[2:10]
print(unknow_iris)
proba = knn.predict_proba(unknow_iris)#[0]
print(proba)
print("setosa \t\t= {}\nversicolor \t= {}\nvirginica \t= {}".format(proba[0], proba[1], proba[2]))

          0         1         2         3
2  1.363608 -0.404683  0.045669  0.198818
3 -2.567370  0.552244 -0.066586  0.028245
4 -2.886608  0.021055 -0.160340 -0.164462
5  1.257516  0.644915 -0.489204 -0.027015
6 -3.013032 -0.344942  0.239463  0.080936
7  0.514276 -1.126970  0.656221  0.134302
8 -2.655977 -0.160104 -0.119918 -0.014939
9  0.645019 -0.310415  0.353463  0.201254
[[0.         0.26666667 0.73333333]
 [1.         0.         0.        ]
 [1.         0.         0.        ]
 [0.         0.66666667 0.33333333]
 [1.         0.         0.        ]
 [0.         0.8        0.2       ]
 [1.         0.         0.        ]
 [0.         0.93333333 0.06666667]]
setosa 		= [0.         0.26666667 0.73333333]
versicolor 	= [1. 0. 0.]
virginica 	= [1. 0. 0.]
